<a id='home_cell'></a>
## BLOG POST - CHI SQUARE 

*Please note I have chosen to use GitHub / Jupyter notebooks in substitute of Tumblr for my blog assignments

Upon analysis of the purpose of the loan, I notice that "debt consolition" has the highest percentage of defaults vs 
non-defaults coming in at 7.13%.  It is also important to note that this category contains the highest number of 
loans overall across all the groups.

Upon analyzing the default vs binned interest rates, I notice that the 10% - 15% group has the highest percentage of 
defaulted loans at 7.30%.  At the lowest and highest bins of the interest rates (0% - 10%, 20% - 25%), it actually has 
the lowest number percentage of loans in default.  The debt to income ratio bin follows the same pattern.  My original 
reserach question - do higher interest rates / debt to income ratios lend to higher defaults.  
For now the answer is no, higher interest rates and dti do NOT lend to higher defaults. 

In addition, for each individual test of the interest rate, the p-value is less 
than 0.05 to indicate that this test is significant and they are associated.

In [7]:
# imports
import pandas
import numpy
import seaborn
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

import scipy.stats

In [8]:
# =====================
# 2. DATA IMPORT 
# =====================
# set working directory
directory = '//Project/OneDrive/Coursera/Wesleyan_Course1/wk1_Assignment/'

pdLoans = pandas.read_csv(directory + 'LoanStats3a.csv', low_memory=False)

# =======================
#  3. DATA TRANSFORMATIONS 
#  ======================= 
# Convert the data elements - just doing in 
pdLoans["int_rate_d"] = pandas.to_numeric(pdLoans["int_rate"].str.replace('%',''), errors='coerce')
pdLoans["revol_util_d"] = pandas.to_numeric(pdLoans["revol_util"].str.replace('%',''), errors='coerce')

# divide by 100 to get the true percentage
pdLoans["int_rate_d"] = pdLoans["int_rate_d"] / 100.00
pdLoans["revol_util_d"] = pdLoans["revol_util_d"] / 100.00

# Convert and Bin the data
pdLoans['int_rate_bin'] = pandas.cut(pdLoans.int_rate_d, [0.0,0.10,0.15,0.20,0.25,0.30])
pdLoans['dti_bin'] = pandas.cut(pdLoans.dti, [0,1,10,20,30])
pdLoans['revol_util_bin'] = pandas.cut(pdLoans.revol_util_d, [0,0.25,0.5,0.75,1])

# Add a veriable for recoveries / collection_recovery_fee.  
# If either is greather than 0, then assign a 1 to indicate a default
# Create a function to Map
def set_Defaulted (row):
    if row['recoveries'] > 0:
        return 1
    if row['collection_recovery_fee'] > 0:
        return 1
    else:
        return 0 

pdLoans['defaulted'] = pdLoans.apply(lambda row: set_Defaulted(row), axis=1)

# Here split out separate date set for the defaulted loans - for analayis use later
pdLoansDefaulted = pdLoans[pdLoans['defaulted'] == 1]

## binned data analysis - counts and percentages 
<a id='binn_analysis'></a>

In [9]:
## For this excercise - I can use the defaulted variable against the %'s above
# defaulted
# int_rate_d, int_rate_bin
# dti
# revol_util_d, revol_util_bin

In [10]:
# contingency count/table of observed counts
print('--== Contingency table of "defaulted" / "interest rate" bins ==--')
ct1 = pandas.crosstab(pdLoans['defaulted'], pdLoans['int_rate_bin'])
print(ct1)

print('')
print('--== Contingency table of "defaulted" / "debt to income" bins ==--')
ct2 = pandas.crosstab(pdLoans['defaulted'], pdLoans['dti_bin'])
print(ct2)

print('')
print('--== Contingency table of "defaulted" / "revolving utilization" bins ==--')
ct3 = pandas.crosstab(pdLoans['defaulted'], pdLoans['revol_util_bin'])
print(ct3)

--== Contingency table of "defaulted" / "interest rate" bins ==--
int_rate_bin  (0.0, 0.1]  (0.1, 0.15]  (0.15, 0.2]  (0.2, 0.25]
defaulted                                                      
0                  11735        17284         6550          619
1                    858         3076         2093          320

--== Contingency table of "defaulted" / "debt to income" bins ==--
dti_bin    (0, 1]  (1, 10]  (10, 20]  (20, 30]
defaulted                                     
0             791    11299     17125      6807
1              97     1703      3126      1381

--== Contingency table of "defaulted" / "revolving utilization" bins ==--
revol_util_bin  (0.0, 0.25]  (0.25, 0.5]  (0.5, 0.75]  (0.75, 1.0]
defaulted                                                         
0                      8358         9594         9546         7710
1                       911         1482         1867         1880


In [11]:
# For debugging only
# After p-values above being 0, I want to take a closer look at the just the variables to be analyzed 
print('--== count of defaulted loans ==--')
print('')
pdSubSetVariables = pdLoans[['int_rate_d','dti','revol_util_d','defaulted']]
subVariablesCount = pdSubSetVariables.groupby('defaulted').count()
print(subVariablesCount)

--== count of defaulted loans ==--

           int_rate_d    dti  revol_util_d
defaulted                                 
0               36188  36188         36120
1                6347   6347          6325


In [12]:
print('')
print('--== percent of defaulted loans vs Interest Rate (binned) ==--')
print('')
colsum = ct1.sum(axis=0)
colpct = ct1/colsum
print(colpct)

print('')
print('--== percent of defaulted loans vs Debt to Income (binned) ==--')
print('')
colsum = ct2.sum(axis=0)
colpct = ct2/colsum
print(colpct)

print('')
print('--== percent of defaulted loans vs Revolving Utilization (binned) ==--')
print('')
colsum = ct3.sum(axis=0)
colpct = ct3/colsum
print(colpct)


--== percent of defaulted loans vs Interest Rate (binned) ==--

int_rate_bin  (0.0, 0.1]  (0.1, 0.15]  (0.15, 0.2]  (0.2, 0.25]
defaulted                                                      
0               0.931867     0.848919     0.757839     0.659212
1               0.068133     0.151081     0.242161     0.340788

--== percent of defaulted loans vs Debt to Income (binned) ==--

dti_bin      (0, 1]  (1, 10]  (10, 20]  (20, 30]
defaulted                                       
0          0.890766  0.86902  0.845637  0.831339
1          0.109234  0.13098  0.154363  0.168661

--== percent of defaulted loans vs Revolving Utilization (binned) ==--

revol_util_bin  (0.0, 0.25]  (0.25, 0.5]  (0.5, 0.75]  (0.75, 1.0]
defaulted                                                         
0                  0.901715     0.866197     0.836415     0.803962
1                  0.098285     0.133803     0.163585     0.196038


## CHI SQUARE 
<a id='chi_sq_analysis'></a>

In [13]:
# chi-square
print('chi-square value, p value, expected counts')
print('')
print('--== percent of defaulted loans vs Interest Rate (binned) ==--')
cs1 = scipy.stats.chi2_contingency(ct1)
print(cs1)
print('')

print('--== percent of defaulted loans vs Debt to Income (binned) ==--')
cs2 = scipy.stats.chi2_contingency(ct2)
print(cs2)
print('')

print('--== percent of defaulted loans vs Revolving Utilization (binned) ==--')
cs3 = scipy.stats.chi2_contingency(ct3)
print(cs3)
print('')


chi-square value, p value, expected counts

--== percent of defaulted loans vs Interest Rate (binned) ==--
(1512.2954834438528, 0.0, 3, array([[ 10713.89406371,  17321.91559892,   7353.30631245,    798.88402492],
       [  1879.10593629,   3038.08440108,   1289.69368755,    140.11597508]]))

--== percent of defaulted loans vs Debt to Income (binned) ==--
(73.927853726234588, 6.1499900627480792e-16, 3, array([[   755.68844055,  11064.70845047,  17233.6110468 ,   6967.99206218],
       [   132.31155945,   1937.29154953,   3017.3889532 ,   1220.00793782]]))

--== percent of defaulted loans vs Revolving Utilization (binned) ==--
(395.69835504408053, 1.8919495251238126e-85, 3, array([[ 7892.59340234,  9431.26168134,  9718.2186321 ,  8165.92628422],
       [ 1376.40659766,  1644.73831866,  1694.7813679 ,  1424.07371578]]))



In [14]:
# chi-square
print('chi-square value, p value, expected counts')


chi-square value, p value, expected counts


## multiple chi_square tests

In [15]:
# I have done interest rate alot.  I am going to use revolving utilization this time
print('')
print('--== Contingency table of "defaulted" / "revolving utilization" bins ==--')
ct3 = pandas.crosstab(pdLoans['defaulted'], pdLoans['revol_util_bin'])
print(ct3)


print('')
print('--== percent of defaulted loans vs Revolving Utilization (binned) ==--')
print('')
colsum = ct3.sum(axis=0)
colpct = ct3/colsum
print(colpct)


cs3 = scipy.stats.chi2_contingency(ct3)
print(cs3)
print('')


--== Contingency table of "defaulted" / "revolving utilization" bins ==--
revol_util_bin  (0.0, 0.25]  (0.25, 0.5]  (0.5, 0.75]  (0.75, 1.0]
defaulted                                                         
0                      8358         9594         9546         7710
1                       911         1482         1867         1880

--== percent of defaulted loans vs Revolving Utilization (binned) ==--

revol_util_bin  (0.0, 0.25]  (0.25, 0.5]  (0.5, 0.75]  (0.75, 1.0]
defaulted                                                         
0                  0.901715     0.866197     0.836415     0.803962
1                  0.098285     0.133803     0.163585     0.196038
(395.69835504408053, 1.8919495251238126e-85, 3, array([[ 7892.59340234,  9431.26168134,  9718.2186321 ,  8165.92628422],
       [ 1376.40659766,  1644.73831866,  1694.7813679 ,  1424.07371578]]))

